# Ejercicio clase 5: CNN-score
El objetivo es armar una arquitectura tipo CNN que identifique el dataset MNIST, tal que logre el mejor score dado por:

$$ score = \frac{1}{log_{10}(n\_parameter)} * \frac{10}{n\_epochs}*test\_acc*n\_layers$$

donde:
*   `test_acc`: valor de accuracy.
*   `n_parameter`: cantidad de parámetros generados por la red.
*   `n_layers`: cantidad de capas empleadas (conv + activacion + pooling = 1 capa).
*   `n_epochs` de entrenamiento usadas.

El modelo se deberá ajustar a los siguientes parámetros:

*   train: 80%, validation: 10%, test: 10% (los datos serán dados así todos usan el mismo set para cada grupo. Están en el github el curso).
*   capa final de salida será una softmax de 10 elementos.
*   coss_function será `CrossEntropyLoss`.

## Librerías necesarias

In [1]:
import torch
import torchvision
import numpy as np
import pickle
import random
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import ToTensor

## Configuración del _device_

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

## Resolución

#### 1. Carga de dataset

In [3]:
# Lectura y carga de los dataset en formato pickle

data = ['train.pkl', 'train_label.pkl', 'test.pkl', 'test_label.pkl', 'val.pkl', 'val_label.pkl']
# lista para acumular los dataset como array de numpy
data_np = []

for i in range(len(data)):
    DATASET_PATH = './data/' + data[i]
    mask_data = open(DATASET_PATH,'rb')
    data_np.append(pickle.load(mask_data))
    mask_data.close()


In [4]:
# Armado de cada dataset como array de numpy. Verificación de tipos de objetos y dimensiones
# Dataset train
train_np = data_np[0]
print("Dataset train_np. Tipo:", type(train_np),  train_np.dtype, ", Dimensiones: ", train_np.shape)

# Dataset train_label
train_label_np = data_np[1]
print("Dataset train_label_np. Tipo:", type(train_label_np), train_label_np.dtype,  ", Dimensiones: ", train_label_np.shape)

# Dataset test
test_np = np.array(data_np[2])
print("Dataset test_np. Tipo:", type(test_np), test_np.dtype, ", Dimensiones: ", test_np.shape)

# Dataset test_label
test_label_np = np.array(data_np[3])
print("Dataset test_label_np. Tipo:", type(test_label_np), test_label_np.dtype, ", Dimensiones: ", test_label_np.shape)

# Dataset validation
val_np = np.array(data_np[4])
print("Dataset validation_np. Tipo:", type(val_np), val_np.dtype, ", Dimensiones: ", val_np.shape)

# Dataset validation_label
val_label_np = np.array(data_np[5])
print("Dataset val_label. Tipo:", type(val_label_np), val_label_np.dtype, ", Dimensiones: ", val_label_np.shape)


Dataset train_np. Tipo: <class 'numpy.ndarray'> uint8 , Dimensiones:  (56000, 28, 28)
Dataset train_label_np. Tipo: <class 'numpy.ndarray'> float64 , Dimensiones:  (56000,)
Dataset test_np. Tipo: <class 'numpy.ndarray'> uint8 , Dimensiones:  (7000, 28, 28)
Dataset test_label_np. Tipo: <class 'numpy.ndarray'> int64 , Dimensiones:  (7000,)
Dataset validation_np. Tipo: <class 'numpy.ndarray'> uint8 , Dimensiones:  (7000, 28, 28)
Dataset val_label. Tipo: <class 'numpy.ndarray'> int64 , Dimensiones:  (7000,)


Creo la clase para crear el dataset

In [5]:
class CustomDataset(Dataset):

    def __init__(self, X_Train, y_Train, transform):
        self.X_Train = X_Train
        self.y_Train = y_Train
        self.transform = transform

    def __len__(self):
        return len(self.X_Train)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        X = self.X_Train[idx]
        y = self.y_Train[idx]

        if self.transform:
            X = self.transform(X)
            y = self.transform(y)

        return X, y

In [6]:
# Creo los dataset
train_dataset = CustomDataset(train_np, train_label_np, transform=transforms.Compose([transforms.ToTensor()]))

# test_dataset = CustomDataset(test_np, test_label_np, transform=transforms.Compose([transforms.ToTensor()]))
# val_dataset = CustomDataset(val_np, val_label_np, transform=trarnsforms.Compose([transforms.ToTensor()]))

In [7]:
# Preparación de batch para entrenamiento
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True) 
# test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)
# val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)

In [8]:
# Display image and label.
train, train_labels = next(iter(train_dataloader))
# print(f"Feature batch shape: {train_features.size()}")
# print(f"Labels batch shape: {train_labels.size()}")
# img = train_features[0].squeeze()
# label = train_labels[0]
# plt.imshow(img, cmap="gray")
# plt.show()
# print(f"Label: {label}")

TypeError: pic should be PIL Image or ndarray. Got <class 'numpy.float64'>